# Linear Modeling

The goal of this lab is to design a class for linear regression, using no classes or functions from Scikit-Learn.

Ensure your class satisfies:
- Includes a class method to fit the model to a Pandas dataframe $X$ and a Pandas series $y$
- Includes a class method to solve for the optimal coefficients
- Includes a class method to make predictions, given a new matrix $\hat{X}$
- Does not invert any matrices explicitly. Instead, solve the normal equations using `np.lingalg.solve`.
- It can be instructed to automatically perform a train-test split and return performance metrics on the test set.
- It can provide metrics including SSE, MSE, RMSE, and $R^2$.

Before programming your class, consider the following questions and record the answers:
- How does your class handle categorical data? How does Sci-kit do it?
- How does your class handle missing data? How does Sci-kit do it?
- Does your class have any methods for creating polynomial expansions or otherwise transforming data? How does Sci-kit do it?
- How does your class handle the bias/intercept/constant? How does Sci-kit do it?
- What output do you automatically provide to the user? Why? How does Sci-kit do it?
- Are you including any tools for statistical inference? How does Sci-kit do it?

In order to measure how long it takes to run code, you can `import time`, and 

```
start = time.time()
<expressions and code>
finish = time.time()
runtime = start-finish
```

For the `heart_hw.csv` and `cars_hw.csv` data in the assignment folder, run some regressions and compare the performance of your class with Sci-Kit's linear regression model. Do you get the same answers for the optimal coefficients, SSE, and $R^2$? Which one runs faster?




Categorical Data

- My class:  need to manually encode categories (e.g., using pd.get_dummies).

- Scikit-Learn: Automatically handles one-hot encoding when combined with ColumnTransformer and OneHotEncoder.


Missing Data

- My class:  must manually handle it (drop rows or fill using imputation).

- Scikit-Learn: Can use SimpleImputer and Pipeline to handle this cleanly.


Polynomial Features

- My class:  can implement a method to manually expand features: e.g., square or interaction terms.
- Scikit-Learn: Uses PolynomialFeatures.


Intercept/Bias Term

- My class: Add a column of 1s to X manually.
- Scikit-Learn: Handles it automatically unless fit_intercept=False.


Automatic Output

- My class: Include metrics like SSE, MSE, RMSE, and R² after .fit().
- Scikit-Learn: Doesn't show metrics automatically—you compute them separately.


Statistical Inference

- My class: Not included unless you implement t-tests or confidence intervals.
- Scikit-Learn: Doesn't support inference either—use statsmodels for that.

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time

class LinearRegressionCustom:
    """Minimal linear-regression class with normal-equation solver."""
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

    # ---------- core API ----------
    def fit(self, X, y):
        """Estimate coefficients β by solving (XᵀX)β = Xᵀy."""
        X = np.asarray(X, dtype=np.float64)  # Ensure X is numeric
        y = np.asarray(y, dtype=np.float64)  # Ensure y is numeric

        # prepend a column of 1s for the intercept
        X = np.c_[np.ones(X.shape[0]), X]

        # Solve for the coefficients using the normal equation
        beta = np.linalg.solve(X.T @ X, X.T @ y)
        self.intercept_, self.coef_ = beta[0], beta[1:]

    def predict(self, X):
        """Return ŷ for a new design matrix."""
        X = np.asarray(X, dtype=np.float64)  # Ensure X is numeric
        X = np.c_[np.ones(X.shape[0]), X]
        return X @ np.r_[self.intercept_, self.coef_]

    # ---------- helpers ----------
    @staticmethod
    def performance_metrics(y_true, y_pred):
        """SSE, MSE, RMSE, R²."""
        sse = np.sum((y_true - y_pred) ** 2)
        mse = sse / len(y_true)
        rmse = np.sqrt(mse)
        r2 = 1 - sse / np.sum((y_true - np.mean(y_true)) ** 2)
        return sse, mse, rmse, r2


# ---------------- main workflow ----------------
if __name__ == "__main__":
    start = time.time()

    # 1. load & clean ---------------------------------------------------------
    df = pd.read_csv("C:/Users/bridget/linearModels/assignment/data/heart_hw.csv")

    # drop the index column
    df = df.drop(columns=["Unnamed: 0"])

    # 2. feature / target split ----------------------------------------------
    y = df["age"]                                # target
    X = df.drop(columns=["age"])                 # predictors

    # one-hot encode the categorical ‘transplant’ column
    X = pd.get_dummies(X, drop_first=True)       # → adds ‘transplant_treatment’

    # 3. train / test split ---------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )

    # 4. fit & evaluate -------------------------------------------------------
    model = LinearRegressionCustom()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    SSE, MSE, RMSE, R2 = model.performance_metrics(y_test, y_pred)

    finish = time.time()

    # 5. results --------------------------------------------------------------
    print("Coefficients :", model.coef_)
    print("Intercept    :", model.intercept_)
    print(f"SSE={SSE:.4f}  MSE={MSE:.4f}  RMSE={RMSE:.4f}  R²={R2:.4f}")
    print(f"Execution time: {finish - start:.3f} s")


Coefficients : [-6.96648535  4.77333743]
Intercept    : 42.53092647832953
SSE=1374.8462  MSE=65.4689  RMSE=8.0913  R²=-0.1964
Execution time: 0.038 s


In [12]:
from sklearn.linear_model import LinearRegression as SklearnLR
import time

# 1. Sklearn model regression ---------------------------------------------
start_sklearn = time.time()

# Create and fit the Sklearn model
sklearn_model = SklearnLR()
sklearn_model.fit(X_train, y_train)

# Make predictions and calculate performance
sklearn_y_pred = sklearn_model.predict(X_test)
sklearn_SSE, sklearn_MSE, sklearn_RMSE, sklearn_R2 = model.performance_metrics(y_test, sklearn_y_pred)

finish_sklearn = time.time()

# 2. Display Sklearn results ----------------------------------------------
print("Sklearn Coefficients:", sklearn_model.coef_)
print("Sklearn Intercept   :", sklearn_model.intercept_)
print(f"Sklearn Performance: SSE={sklearn_SSE}, MSE={sklearn_MSE}, RMSE={sklearn_RMSE}, R²={sklearn_R2}")
print(f"Sklearn Execution time: {finish_sklearn - start_sklearn:.3f} s")


Sklearn Coefficients: [-6.96648535  4.77333743]
Sklearn Intercept   : 42.53092647832953
Sklearn Performance: SSE=1374.8462045478798, MSE=65.46886688323237, RMSE=8.091283389131316, R²=-0.19641017302774233
Sklearn Execution time: 0.020 s


Observations:
- Coefficients and Intercept: Both models yield identical coefficients and intercepts, which is expected since they are solving the same problem using the normal equation.

- Performance Metrics: The performance metrics (SSE, MSE, RMSE, and R²) are essentially the same, which confirms that both models are making the same predictions.

- Execution Time: Scikit-learn is slightly faster (0.020 s vs. 0.038 s), Which is probably due to optimizations in the library's implementation of linear regression.